# Benchmark of Symmetry-Based Subgraph Matching Algorithm
---
### Basic Information
**Description:** This script benchmarks the runtime of a symmetry-based subgraph matching algorithm and compares its efficiency with common subgraph matching algorithms such as VF2 and VF2++. Particularly, we consider a target graph with an octagonal structure.

## Load Packages and Define Basic Functions

In [1]:
# Load packages
from rustworkx import *
from rustworkx.visualization import mpl_draw

from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_dag

import networkx as nx
import rustworkx as rx
import rustworkx.generators
import numpy as np
import time

%run functions/functions_set_graph.ipynb # import graph generator
%run functions/functions_precompilation.ipynb # import precompiling functions
%run functions/functions_subgraph_matching.ipynb # import subgraph matching functions
%run functions/functions_circuit_mapping.ipynb # import functions for circuit mapping
%run functions/functions_test_circuits.ipynb # import the predefined circuits

## Benchmark Runtime of Subgraph Matching Algorithms

### Quantum Hardware Architecture: Octagonal

In [ ]:
# set quantum circuit
circuit_width = 5
oracle = dj_oracle(circuit_width - 1)
input_circ = deutsch_jozsa_algorithm(oracle, circuit_width - 1) # input circuit

# define a reduced hardware coupling map for precompilation
reduced_coupling_graph, node_dict, node_dict_reverse = set_octagonal_graph(5, 5)
directional_edge_list = list(reduced_coupling_graph.edge_list())
bidirectional_edge_list = []
for edge in directional_edge_list:
    bidirectional_edge_list.append((edge[0], edge[1]))
    bidirectional_edge_list.append((edge[1], edge[0]))
reduced_coupling_map = CouplingMap(bidirectional_edge_list)
# reduced_coupling_map.draw()

# precompilation, find an initial mapping of the circuit onto the hardware
circ = precompile(circuit=input_circ, cmap=reduced_coupling_map)
graph_b = circuit_interaction_graph(circ)
mpl_draw(graph_b, with_labels=True)

In [ ]:
input_circ.draw(output='mpl')

In [ ]:
circ.draw(output='mpl')

In [ ]:
# benchmarking runtime for octagonal coupling graph

# parameters
graph_type = 'octagonal'
size_target_graph_list = np.arange(10, 110, 5) # number of qubits in a row/column of the target graph
# size_target_graph_list = np.arange(20, 75, 5) # number of qubits in a row/column of the target graph

octagonal_VF2_runtime_list = []
octagonal_VF2pp_runtime_list = []
octagonal_node_count = []
octagonal_search_space_runtime_list = []

for size_target_graph in size_target_graph_list:
    # define the graph of the circuit and the hardware
    graph_a, node_dict, node_dict_reverse = set_octagonal_graph(size_target_graph, size_target_graph)

    # number of qubits
    num_qubit_hardware = graph_a.num_nodes() # number of qubits in the hardware
    octagonal_node_count.append(num_qubit_hardware)

    # measure runtime of VF2 for matching these two graphs
    start_time = time.time()
    layouts = mapping_VF2(graph_a, graph_b, circ)
    runtime = time.time() - start_time
    print(f'Qubits in the hardware: {num_qubit_hardware}, qubits in the circuit: {circ.num_qubits}')
    print(f'VF2 runtime: {runtime} seconds')
    octagonal_VF2_runtime_list.append(runtime)

    # measure runtime of VF2++ for matching these two graphs
    start_time = time.time()
    layouts = mapping_VF2pp(graph_a, graph_b, circ)
    runtime = time.time() - start_time
    print(f'VF2++ runtime: {runtime} seconds')
    octagonal_VF2pp_runtime_list.append(runtime)

    # measure runtime of the octagonal-specific subgraph matching algorithm for the same task as above
    start_time = time.time()
    center_generating_set = generating_set([size_target_graph, size_target_graph], graph_type, node_dict_reverse)
    # layouts = symmetry_based_subgraph_matching(graph_a, graph_b, node_dict, node_dict_reverse, center_generating_set)
    layouts = symmetry_based_subgraph_matching_optimized(graph_a, graph_b, node_dict, node_dict_reverse, center_generating_set, graph_type, [size_target_graph, size_target_graph])
    runtime = time.time() - start_time
    print(f'Symmetry-Based Subgraph Matching runtime: {runtime} seconds')
    octagonal_search_space_runtime_list.append(runtime)
    print('\n')

# save results
np.save(f'data/subgraph_matching_{graph_type}.npy', [np.array(octagonal_node_count), np.array(octagonal_VF2_runtime_list), np.array(octagonal_VF2pp_runtime_list), np.array(octagonal_search_space_runtime_list)])

In [ ]:
import matplotlib.pyplot as plt
import matplotlib

# plotting parameters
linewidth = 2
markersize = 5
marker = 'o'
colors = ['orangered', 'royalblue', 'purple']
labels = ['VF2', 'VF2++', 'SBSM (this work)']
plt.rcParams.update({'font.size': 10}) # font size
matplotlib.rc('font', family='Arial') # font style
fig_width = 8.6 # width of figure in cm
fig_height = 8 # height of figure in cm
figsize = (fig_width/2.54, fig_height/2.54)
dpi = 1200

# plot runtime versus number of qubits
x = octagonal_node_count
y1 = octagonal_VF2_runtime_list
y2 = octagonal_VF2pp_runtime_list
y3 = octagonal_search_space_runtime_list

plt.figure(figsize=figsize)
plt.plot(x, y1, label=labels[0], linewidth=linewidth, marker=marker, markersize=markersize, color=colors[0])
plt.plot(x, y2, label=labels[1], linewidth=linewidth, marker=marker, markersize=markersize, color=colors[1])
plt.plot(x, y3, label=labels[2], linewidth=linewidth, marker=marker, markersize=markersize, color=colors[2])

plt.title(f'search time for finding all isomorphisms')
plt.xlabel('number of qubits in the hardware')
plt.ylabel('runtime (seconds)')

plt.legend()
plt.tight_layout()

plt.savefig(f'figures/subgraph_matching_runtime_{graph_type}_hardware.png', dpi=dpi)
plt.show()